In [69]:
import psycopg2
import pymongo
import datetime
import base64
from decimal import Decimal

In [70]:
conn_string = "host='localhost' dbname='dvdrental' user='postgres'"

In [71]:
conn = psycopg2.connect(conn_string)
cursor = conn.cursor()

In [72]:
table_names = []
cursor.execute("""
    SELECT table_name
    FROM information_schema.tables
    WHERE table_schema='public'
    AND table_type='BASE TABLE';
    """)
for table in cursor.fetchall():
    table_names.append(table[0])

In [73]:
table_columns = {}
for name in table_names:
    cursor.execute("SELECT column_name FROM information_schema.columns WHERE table_schema = 'public' AND table_name = '" + name + "';")
    table_columns[name] = []
    for column in cursor.fetchall():
        table_columns[name].append(column[0])
    

In [74]:
def select_all_from(table, cur):
    cur.execute('SELECT * FROM ' + table + ';')
    return cur.fetchall()

In [75]:
def get_dict_list(table, cur):
    columns = table_columns[table]
    res = []
    l = select_all_from(table, cur)
    for rec in l:
        temp = {}
        for i in range(len(columns)):
            if isinstance(rec[i], datetime.date):
                temp[columns[i]] = datetime.datetime.combine(rec[i], datetime.time.min)
            elif isinstance(rec[i], memoryview):
                temp[columns[i]] = str(base64.b64encode(rec[i]))
            elif isinstance(rec[i], Decimal):
                temp[columns[i]] = float(rec[i])
            else:
                temp[columns[i]] = rec[i]
        res.append(temp)
    return res

In [76]:
client = pymongo.MongoClient('localhost', 27017)
db = client.db
for col in db.list_collection_names():
    db[col].drop()

In [77]:
for table in table_names:
    col = db[table]
    print(table)
    col.insert_many(get_dict_list(table, cursor))

actor
store
address
category
city
country
customer
film_actor
film_category
inventory
language
rental
staff
payment
film
